In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.pipeline import Pipeline
# from xgboost import XGBRegressor

import math
import time
import xgboost as xgb
import gc
import pickle
from numpy import array

In [2]:
with open('train2_1124', 'rb') as f:
    train_all = pickle.load(f)
with open('test2_1124', 'rb') as f:
    test_all = pickle.load(f)

In [3]:
display(train_all)

,building_id,meter,meter_reading,square_feet,floor_count,has_year,has_floorcnt,age,precip_true_1_hr,air_temperature,...,hour_22,hour_23,wind_direction_0,wind_direction_1,wind_direction_2,wind_direction_3,wind_direction_4,wind_direction_5,wind_direction_6,wind_direction_7
0,0,0,0.000000,7432,0,1,0,9,0,25.0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0.000000,2720,0,1,0,13,0,25.0,...,0,0,1,0,0,0,0,0,0,0
2,2,0,0.000000,5376,0,1,0,26,0,25.0,...,0,0,1,0,0,0,0,0,0,0
3,3,0,0.000000,23685,1,1,0,15,0,25.0,...,0,0,1,0,0,0,0,0,0,0
4,4,0,0.000000,116607,6,1,0,42,0,25.0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20216095,1444,0,8.750000,19619,2,1,0,103,0,1.7,...,0,1,0,0,0,0,1,0,0,0
20216096,1445,0,4.825000,4298,0,0,0,54,0,1.7,...,0,1,0,0,0,0,1,0,0,0
20216097,1446,0,0.000000,11265,1,1,0,20,0,1.7,...,0,1,0,0,0,0,1,0,0,0
20216098,1447,0,159.574997,29775,2,1,0,16,0,1.7,...,0,1,0,0,0,0,1,0,0,0


In [4]:
submission = pd.read_csv('files/test.csv')

In [5]:
submission.insert(0, 'predicted', 0)
submission.insert(0, 'meter_reading', 0)

In [6]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
#             print("******************************")
#             print("Column: ",col)
#             print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
#             print("dtype after: ",props[col].dtype)
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist


In [7]:
submission, NAlist = reduce_mem_usage(submission)

Memory usage of properties dataframe is : 1908.7647705078125  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  676.0209350585938  MB
This is  35.416670796933424 % of the initial size


In [8]:
print(train_all.shape)
print(test_all.shape)

(20125605, 87)
(41498571, 87)


In [9]:
def evalerror(preds, actual):
    preds[preds<0] = 0
    return math.sqrt(np.square(np.log(preds+1)-np.log(actual+1)).mean())

In [10]:
def learn2(X_train, X_valid, y_train, y_valid):
#     X_train, X_valid, y_train, y_valid = train_test_split(X_full, y, train_size=0.8, test_size=0.2, random_state=0)
    params = {
        'objective' : 'reg:squarederror',
#         'early_stopping_rounds': 10,
        'nthread' : 8,
#         'num_rounds' : 300
        'tree_method': 'gpu_hist',
    }

    plst = params.items()
    dtrain = xgb.DMatrix( data = X_train.values, label = y_train.values)
    dtest = xgb.DMatrix( data = X_valid.values)
#     return (0,0)
    num_rounds = 100
    model = xgb.train(plst, dtrain, num_rounds)

    # predict 
    preds = model.predict(dtest)
    preds[preds<0] = 0
    score = evalerror(y_valid,preds)
    return (score,preds)


In [11]:
def learn_real(X_full, y, X_test):
#     rmsle to rmse
    y = np.log(y+1)
    X_train, X_valid, y_train, y_valid = train_test_split(X_full, y, train_size=0.8, test_size=0.2, random_state=0)
    del X_full,y,X_valid,y_valid
    gc.collect()
    params = {
        'objective' : 'reg:squarederror',
        'early_stopping_rounds': 10,
        'nthread' : 8,
#         'num_rounds' : 300
        'tree_method': 'gpu_hist',
    }
    num_rounds = 500
    plst = params.items()
    dtrain = xgb.DMatrix( data = X_train.values, label = y_train.values)
    del X_train,y_train
    gc.collect()
    model = xgb.train(plst, dtrain, num_rounds)
    # predict 
    del dtrain
    gc.collect()
    l = X_test.shape[0]
    preds = np.array([])
    for i in range(5):
        a = int(l*i/5)
        b = int(l*(i+1)/5)
        X_test2 = X_test.iloc[a:b]
        dtest = xgb.DMatrix( data = X_test2.values)
        preds2 = model.predict(dtest)
        preds = np.concatenate((preds, preds2), axis=None)
#     rmsle to rmse
    preds = np.exp(preds) - 1
    preds[preds<0] = 0
    return preds


In [12]:
def predict(train,test):
    tick = time.time()
    y_validall = []
    y_predsall = []

    y = train['meter_reading']
    train = train.drop('meter_reading', axis=1)
    #   for learn2
#     X_train, X_valid, y_train, y_valid = train_test_split(train, y, train_size=0.8, test_size=0.2, random_state=0)
#     (score,preds) = learn2(X_train, X_valid, y_train, y_valid )

#     y_validall.append(y_valid.values)
#     y_predsall.append(preds)
    
#     y_validall2 = np.concatenate( y_validall, axis=0 )
#     y_predsall2 = np.concatenate( y_predsall, axis=0 )
#     print(evalerror(y_validall2,y_predsall2))
    
#     tock = time.time()
#     print('Time used : {}'.format(tock-tick))
    
    #   for learn_real
    y_row = test['row_id']
    test = test.drop('row_id', axis=1)
    preds = learn_real(train, y, test)
# do not pass

#     X_train, X_valid, y_train, y_valid = train_test_split(train, y, train_size=0.8, test_size=0.2, random_state=0)
#     del train, y
#     gc.collect()
#     params = {
#         'objective' : 'reg:squarederror',
#         'early_stopping_rounds': 10,
#         'nthread' : 4,
# #         'num_rounds' : 300
#         'tree_method': 'gpu_hist',
#     }

#     plst = params.items()
#     dtrain = xgb.DMatrix( data = X_train.values, label = y_train.values)
#     del X_train,y_train
#     gc.collect()
    
#     dtest = xgb.DMatrix( data = test.values)
#     del test
#     gc.collect()
    
#     num_rounds = 120
#     model = xgb.train(plst, dtrain, num_rounds)
#     # predict 
#     preds = model.predict(dtest)
#     preds[preds<0] = 0

#     y_predsall.append(preds)
#     return (0,0)
    tock = time.time()
    print('Time used : {}'.format(tock-tick))
    
    return(y_row,preds)

    # print(y_predsall)
    # with open('y_predsall1105_2', 'wb') as f:
    #     pickle.dump(y_predsall, f)

In [13]:
split_building = [1099]

In [14]:
drop_col = [
'building_id',
'meter',
# 'timestamp',
# 'meter_reading',
# 'site_id',
'cloud_coverage',
# 'precip_depth_1_hr',
# 'sea_level_pressure'
]


In [15]:
gc.collect()

80

In [16]:
def learn_cv(train ,num_rounds):
    tick = time.time()
    y = train['meter_reading']
    train = train.drop('meter_reading', axis=1)
    y = np.log(y+1)
    X_train, X_valid, y_train, y_valid = train_test_split(train, y, train_size=0.8, test_size=0.2, random_state=357)
    del train
    gc.collect()
    params = {
        'objective' : 'reg:squarederror',
        'early_stopping_rounds': 10,
        'nthread' : 8,
#         'num_rounds' : 300
        'tree_method': 'gpu_hist',
    }
#     num_rounds = 500
    plst = params.items()
    dtrain = xgb.DMatrix( data = X_train.values, label = y_train.values)
    model = xgb.train(plst, dtrain, num_rounds)
    y_train = np.exp(y_train) - 1
    y_valid = np.exp(y_valid) - 1
    # predict training test
    del dtrain
    gc.collect()
    
    l = X_train.shape[0]
    preds_train = np.array([])
    for i in range(5):
        a = int(l*i/5)
        b = int(l*(i+1)/5)
        X = X_train.iloc[a:b]
        dtrain = xgb.DMatrix( data = X.values)
        preds2 = model.predict(dtrain)
        preds_train = np.concatenate((preds_train, preds2), axis=None)
    del dtrain
    
    
#     dtrain2 = xgb.DMatrix( data = X_train.values)
#     preds_train = model.predict(dtrain2)
#     del dtrain2
#     gc.collect()
    preds_train = np.exp(preds_train)-1
    preds_train[preds_train<0] = 0
    
    dvalid = xgb.DMatrix( data = X_valid.values)
    preds_valid = model.predict(dvalid)
    preds_valid = np.exp(preds_valid)-1
    preds_valid[preds_valid<0] = 0
    err1 = evalerror(y_train,preds_train)
    err2 = evalerror(y_valid,preds_valid)
#     rmsle to rmse
    tock = time.time()
    print(tock-tick)
    return (err1,err2)


In [17]:
for meter in range(0,1):
    print('Meter {}'.format(meter))
    train_meter = train_all[train_all['meter'] == meter]
    test_meter = test_all[test_all['meter'] == meter]
    train = train_meter[~train_meter['building_id'].isin(split_building)]
    test = test_meter[~test_meter['building_id'].isin(split_building)]
    split_two = 1
    if train_meter.shape[0] == train.shape[0]:
        split_two = 0
    del train_meter,test_meter
    gc.collect()
#     meter_stat = train['meter_reading'].describe()
#     print(meter_stat)
    train.drop(drop_col,axis=1,inplace=True)
    test.drop(drop_col,axis=1,inplace=True)
    e1 = []
    e2 = []
    for num_rounds in range(200,1200,200):
        print('Starting with rounds {}'.format(num_rounds))
        (err1,err2) = learn_cv(train,num_rounds)
        e1.append(err1)
        e2.append(err2)
        print(num_rounds,err1,err2)
    
    
#     split building_no = 1099
#     if split_two == 1 :
#         train = train_all[train_all['building_id'].isin(split_building)]
#         test = test_all[test_all['building_id'].isin(split_building)]
#         train = train[train['meter'] == meter]
#         test = test[test['meter'] == meter]
# #         meter_stat = train['meter_reading'].describe()
# #         print(meter_stat)
#         train.drop(drop_col,axis=1,inplace=True)
#         test.drop(drop_col,axis=1,inplace=True)
#         (row, preds) = predict(train, test)
#         del train, test
#         gc.collect()
#         row = row.values.astype(int)
#         submission.at[row,'predicted'] = 1
#         submission.at[row,'meter_reading'] = preds
#         del row, preds
#         gc.collect()

Meter 0
Starting with rounds 200
77.72621703147888
200 0.631277333681177 0.6311271028519532
Starting with rounds 400
134.54747700691223
400 0.5441871021496058 0.5449815669573495
Starting with rounds 600
191.65978002548218
600 0.5021491861511892 0.5035088028900625
Starting with rounds 800
247.84136033058167
800 0.47634560897628886 0.47858845681985
Starting with rounds 1000
304.7627499103546
1000 0.46015868499957513 0.4630967337235941
